In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
import h2o
from h2o.automl import H2OAutoML
h2o.init()

/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed i

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_144"; Java(TM) SE Runtime Environment (build 1.8.0_144-b01); Java HotSpot(TM) 64-Bit Server VM (build 25.144-b01, mixed mode)
  Starting server from /Users/captunez/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kq/gb2zltdn4314nj1kcw7mds1m0000gn/T/tmp3moas7wc
  JVM stdout: /var/folders/kq/gb2zltdn4314nj1kcw7mds1m0000gn/T/tmp3moas7wc/h2o_captunez_started_from_python.out
  JVM stderr: /var/folders/kq/gb2zltdn4314nj1kcw7mds1m0000gn/T/tmp3moas7wc/h2o_captunez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster version:,3.16.0.4
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_captunez_gytcr5
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
data = {
    'tra': pd.read_csv('input/air_visit_data.csv'),
    'as': pd.read_csv('input/air_store_info.csv'),
    'hs': pd.read_csv('input/hpg_store_info.csv'),
    'ar': pd.read_csv('input/air_reserve.csv'),
    'hr': pd.read_csv('input/hpg_reserve.csv'),
    'id': pd.read_csv('input/store_id_relation.csv'),
    'tes': pd.read_csv('input/sample_submission.csv'),
    'hol': pd.read_csv('input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [3]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [4]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
# data['tra']['dom'] = data['tra']['visit_date'].dt.day
data['tra']['doy'] = data['tra']['visit_date'].dt.dayofyear
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['week'] = data['tra']['visit_date'].dt.week
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date


data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
# data['tes']['dom'] = data['tes']['visit_date'].dt.day
data['tes']['doy'] = data['tes']['visit_date'].dt.dayofyear
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['week'] = data['tes']['visit_date'].dt.week
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [5]:
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)
#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
# tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
# stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

In [6]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(4):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name' +str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))

stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])


In [8]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [7]:
weather_store = pd.read_csv('input/air_store_weather.csv',parse_dates=['calendar_date'])
weather_store = weather_store.rename(columns={'calendar_date':'visit_date'})

In [9]:
train['visit_date'] = pd.to_datetime(train['visit_date'].values)
test['visit_date'] = pd.to_datetime(test['visit_date'].values)
train = pd.merge(train, weather_store, on = ['air_store_id', 'visit_date'])
test = pd.merge(test, weather_store, on = ['air_store_id', 'visit_date'])

In [10]:
newyear_dates = pd.date_range(datetime(2016,1,1),datetime(2016,1,5)).append(pd.date_range(datetime(2016,12,31),datetime(2017,1,5)))
goldweek = pd.date_range(datetime(2016,4,29),datetime(2016,5,5)).append(pd.date_range(datetime(2017,4,29),datetime(2017,5,5)))
train['visit_date'] = pd.to_datetime(train['visit_date'].values)
test['visit_date'] = pd.to_datetime(test['visit_date'].values)
train['newyear'] = train['visit_date'].isin(newyear_dates)
train['goldweek'] = train['visit_date'].isin(goldweek)
test['newyear'] = test['visit_date'].isin(newyear_dates)
test['goldweek'] = test['visit_date'].isin(goldweek)

In [11]:
reserve_features = ['rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
       'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean']
genre_area_features = ['air_genre_name3', 'air_area_name3', 'air_genre_name4',
       'air_area_name4', 'air_genre_name5', 'air_area_name5',
       'air_genre_name6', 'air_area_name6', 'air_genre_name7',
       'air_area_name7', 'air_genre_name8', 'air_area_name8',
       'air_genre_name9', 'air_area_name9']
train = train[[col for col in train.columns if col not in reserve_features and col not in genre_area_features]]

In [12]:
test = test[[col for col in train.columns if col not in reserve_features and col not in genre_area_features]]

In [13]:
# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

train = train.fillna(-999)
test = test.fillna(-999)

train['visitors'] = np.log1p(train['visitors'].values)


#col = [c for c in sub_train if c not in ['id', 'air_store_id', 'visit_date','visitors']]

print('Pre-processing done!')

Pre-processing done!


In [19]:
test.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'doy', 'year', 'month',
       'week', 'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors',
       'median_visitors', 'count_observations', 'air_genre_name',
       'air_area_name', 'latitude', 'longitude', 'air_genre_name0',
       'air_area_name0', 'air_genre_name1', 'air_area_name1',
       'air_genre_name2', 'air_area_name2', 'date_int', 'var_max_lat',
       'var_max_long', 'avg_temperature', 'high_temperature',
       'low_temperature', 'precipitation', 'hours_sunlight', 'solar_radiation',
       'avg_wind_speed', 'avg_vapor_pressure', 'avg_local_pressure',
       'avg_humidity', 'avg_sea_pressure', 'cloud_cover', 'newyear',
       'goldweek', 'lon_plus_lat', 'air_store_id2'],
      dtype='object')

In [20]:


htrain = h2o.H2OFrame(train)
htest = h2o.H2OFrame(test)

htrain.drop(['air_store_id2', 'air_store_id', 'visit_date'])
htest.drop(['air_store_id2', 'air_store_id', 'visit_date'])

x =htrain.columns
y ='visitors'
x.remove(y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
def RMSLE(y_, pred):
    return metrics.mean_squared_error(y_, pred)**0.5

print('Starting h2o autoML model!')  

aml = H2OAutoML(max_runtime_secs = 3350)
aml.train(x=x, y =y, training_frame=htrain, leaderboard_frame = htest)

print('Generate predictions...')
htrain.drop(['visitors'])
preds = aml.leader.predict(htrain)
preds = preds.as_data_frame()
print('RMSLE H2O automl leader: ', RMSLE(train['visitors'].values, preds))

preds = aml.leader.predict(htest)
preds = preds.as_data_frame()

test['visitors'] = preds
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()

Starting h2o autoML model!
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Generate predictions...
deeplearning prediction progress: |███████████████████████████████████████| 100%
RMSLE H2O automl leader:  0.561787387616
deeplearning prediction progress: |███████████████████████████████████████| 100%


KeyError: "['id'] not in index"

In [11]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('input/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

sub_merge['visitors'] = 0.5*sub_merge['visitors_x'] + 0.5*sub_merge['visitors_y']*1.1
sub_merge[['id', 'visitors']].to_csv('submission.csv', index=False)

print('Leaderboard : ', aml.leaderboard)

print(' H2O automl leader performace : ', aml.leader)

Leaderboard :  

model_id,mean_residual_deviance,rmse,mae,rmsle
StackedEnsemble_BestOfFamily_0_AutoML_20180131_103735,7.65459,2.76669,2.6634,1.29455
GBM_grid_0_AutoML_20180131_103735_model_0,7.66804,2.76912,2.66916,1.29599
GBM_grid_0_AutoML_20180131_103735_model_2,7.68041,2.77135,2.67407,1.29732
StackedEnsemble_AllModels_0_AutoML_20180131_103735,7.68825,2.77277,2.67395,1.29726
DRF_0_AutoML_20180131_103735,7.70791,2.77631,2.67752,1.29819
GBM_grid_0_AutoML_20180131_103735_model_1,7.7086,2.77644,2.67898,1.29863
GLM_grid_0_AutoML_20180131_103735_model_0,27.4798,5.24212,3.1067,nan



 H2O automl leader performace :  Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_0_AutoML_20180131_103735
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.18644277334849377
RMSE: 0.43179019598468626
MAE: 0.3173197451179962
RMSLE: 0.136348372482136
R^2: 0.7142062471184123
Mean Residual Deviance: 0.18644277334849377
Null degrees of freedom: 201535
Residual degrees of freedom: 201532
Null deviance: 131475.68969122303
Residual deviance: 37574.93076956204
AIC: 233438.2928835969

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 0.2290249063146388
RMSE: 0.47856546711462455
MAE: 0.351721413170756
RMSLE: 0.14955256817901585
R^2: 0.6490656661157548
Mean Residual Deviance: 0.2290249063146388
Null degrees of freedom: 50571
Residual degrees of freedom: 50568
Null deviance: 33004.18690112252
Residual deviance: 11582.247562143913
AIC: 68987.80816